# **Water Quality using Random Forest Classification - Knightbearr**

# **Workflow**

1. Data Collection
2. Data Cleaning & Checking
3. EDA
4. Splitting Data
5. Oversampling model
6. Modeling
7. Prediction Score


note : Sorry if my english is bad, and sorry if i had a mistake. thanks in advance!

# **Import Libraries**

import the module that we want to use for this research.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import metrics
from imblearn.over_sampling import SMOTE
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
pd.set_option('display.width', 150)
pd.set_option('display.max_columns', 12)
plt.style.use("seaborn-whitegrid")
sns.set_theme(color_codes=True, style='darkgrid', palette='deep', font='sans-serif')

# **Load The Dataset**

Load the dataset that we want to research

In [ ]:
waterData = pd.read_csv('../input/water-potability/water_potability.csv')

# **Clean and Checking the Data**

we must check the data every time we want to make a model, because this is the important thing, if you suddenly meet a bad dataset, wether you want it or not, you must clean the data.

In [ ]:
# Checking the first 5 rows of data
waterData.head()

In [ ]:
# Checking the last 5 rows of data
waterData.tail()

In [ ]:
# Getting the statistical measure info
waterData.describe()

In [ ]:
# Getting the information about the dataset
waterData.info()

In [ ]:
# Chechking the null data
waterData.isnull().sum()

In [ ]:
# Checking the shape of data
waterData.shape

**First**, we overcome the null values in the ph column first, and we know that the **ph values less than 6.5 and more than 8.5 are not suitable** for consumption, and the **range of pH that is within the WHO standard is 6.52 - 6.83** , and also we have obtained statistical info from the data above, that the **mean value at pH is 7** which is still within the scope of water that is safe for drinking by humans, so we will fill this empty ph value with the mean value of this ph to improve our model

In [ ]:
# Create a new variable named meanPh to hold the value of mean in ph
meanPh = waterData['ph'].mean()

# Fill the blank/null data ph with value of meanPh
waterData['ph'] = waterData['ph'].fillna(meanPh)

# Check the null value and print
print(f'Null Value : {waterData.ph.isnull().sum()}\n')

# Check the data ph
waterData['ph']

**Now**, we overcome the null value in sulfate, and we know that **Sulfate is one of the important ions in the availability of water** because of its important effect for humans when it is available in large quantities. **The maximum sulfate limit in water is about 250 mg/L** for water for human consumption. if we see the statistics info above, **the mean of the Sulfate is 333mg/L**, which mean, that's not good for consumption, and we know that, why the mean value is 333ml/L? this happens because there are so many null values.

In [ ]:
# Created a new variable named waterSulfate tohold the value
waterSulfate = (waterData.Sulfate.mean() - waterData.Sulfate.min())

# Fill the blank/null dataSulfate with the value of waterSulfate
waterData['Sulfate'] = waterData['Sulfate'].fillna(waterSulfate)

# Check the null value and print
print(f'Null Value : {waterData.Sulfate.isnull().sum()}')

# Check the data Sulfate
waterData['Sulfate']

**And last**, we deal with Trihalomethanes, what are Trihalomethanes ? **Trihalomethanes (THMs) are among the most dangerous chemical compounds** that find their way into the water supply. The concentration of THM in drinking water varies according to the level of organic matter in the water, the amount of chlorine required to treat the water, and the temperature of the treated water. **THM levels up to 80 ppm are considered safe** in drinking water. and if we see the statistical measure above, we can see the max value of thm is so high almost doubled value of thm, which that means is bad for consume.


In [ ]:
# Created a new variable named waterTrihalomethanes to hold the value
waterTrihalomethanes = waterData.Trihalomethanes.mean()

# Fill the blank/null data Trihalomethanes with the value of waterTrihalomethanes
waterData['Trihalomethanes'] = waterData['Trihalomethanes'].fillna(waterTrihalomethanes)

# Check the null value and print
print(f'Null Value : {waterData.Trihalomethanes.isnull().sum()}')

# Check the data Sulfate
waterData['Trihalomethanes']

**Checking again**

In [ ]:
# Checking data and print
print(f'Isnull ? :\n{waterData.isnull().sum()}\n')

# Checking data and print
print(f'Is all the data is True ? :\n{waterData.any()}\n')

# Count the value and print
print(f'Potability :\n{waterData.Potability.value_counts()}\n')

# Check the shape and print
print(f'Data Shape : {waterData.shape}')

In [ ]:
# Getting the statistical measure info
waterData.describe()

# **EDA**

analyze and investigate data sets and summarize their main characteristics, often employing data visualization methods.

In [ ]:
# Make a correlation data to knowing Value Strength and Direction of Linear Relationship
corr = waterData.corr()
corr

In [ ]:
# Checking the structure of the data
waterData.sample(11, random_state=1).T

In [ ]:
# Constructing a heatmap to understand the correlation
plt.figure(figsize=(12, 10))
sns.heatmap(corr, cbar=True, square=True, fmt='.1f', annot=True, annot_kws={'size': 8}, cmap='YlGnBu')
plt.plot()

In [ ]:
# Create Regression Plot
sns.regplot(x=waterData.Hardness, y=waterData.Potability, data=waterData)
plt.show()

In [ ]:
# Create a histogram plot
waterData.hist(figsize=(12,12))
plt.show()

**Coefficient of Variation**

The coefficient of variation is a measure of variance that can be used to compare a data distribution that has different units.

* **The higher the Coefficient of Variation** = the wider the data you have compared to the average data (more difficult to predict)
* **The Lower Coefficient of Variation** = The narrower the data you have compared to the Average data (Easier to predict)

In [ ]:
# Coefficient of Potability
covPota = ((waterData['Potability'].std()/waterData['Potability'].mean()) * 100)
print(f'Coefficient Of Variation Potability : {covPota}%')

as you can see the output above, the coefficient of variation is so high, which mean, is so difficult to predict 

In [ ]:
# Getting the Mutual Information about the data
X = waterData.copy()
y = X.pop('Potability')

# All discrete features should now have integer dtypes
discreateFeatures = X.dtypes == int

In [ ]:
# Make a function
def makeMiScores(X, y, discreateFeatures):
    miScores = mutual_info_regression(X, y, discrete_features=discreateFeatures)
    miScores = pd.Series(miScores, name='MI Scores', index=X.columns)
    miScores = miScores.sort_values(ascending=False)
    return miScores

miScores = makeMiScores(X, y, discreateFeatures)
miScores # show a features with their MI scores

In [ ]:
# And now bar plot to make comparisons easier
def plotMiScores(scores):
    scores = scores.sort_values(ascending=True)
    width = np.arange(len(scores))
    ticks = list(scores.index)
    plt.barh(width, scores)
    plt.yticks(width, ticks)
    plt.title("Mutual Information Scores")

# Figuring the plot and plotting
plt.figure(dpi=100, figsize=(6, 3))
plotMiScores(miScores)

Data visualization is a great follow-up to a utility ranking. as we can see the **Hardness, Conductivity, Organic, Turbidity, Solids, and ph have a mutual information** with Potability.

# **Splitting the Data**

divide the data and split it using train test split module from sklearn.

In [ ]:
# Divide the data
X = waterData.drop(['Potability'], axis=1)
y = waterData['Potability']

In [ ]:
# Splitting Data
trainX, testX, \
trainY, testY = train_test_split(X, y, 
                                test_size=.1, 
                                random_state=12)

In [ ]:
# Checking the Target
trainY.value_counts()

In [ ]:
# Checking the Target
testY.value_counts()

# **Upsampling the Target**

upsampling the target using SMOTE, upsampling the target, because we can see the portability and not portability data is have a  huge difference.

In [ ]:
sm = SMOTE(random_state=12)
trainXres, trainYres = sm.fit_resample(trainX, trainY)

# **Train and Fit the model**

Train and fit the model using **RandomForestClassifier** Algorithm.

In [ ]:
model = RandomForestClassifier(n_estimators=500, min_samples_leaf=1, random_state=12)
model = model.fit(trainXres, trainYres)

**Pipelines** are a simple way to keep your data preprocessing and modeling code organized. Specifically, a pipeline bundles preprocessing and modeling steps so you can use the whole bundle as if it were a single step.

In [ ]:
myPipeline = Pipeline(steps=[('model', model)])

In [ ]:
# Train Predict
predTrain = myPipeline.predict(trainXres)
print(metrics.classification_report(trainYres, predTrain))

In [ ]:
# Test Predict
predTest = myPipeline.predict(testX)
print(metrics.classification_report(testY, predTest))